### CP3 - Data Science

- ### Fabrício Saavedra - RM97631
- ### Guilherme Akio - RM98582
- ### Matheus Motta - RM550352

### Base: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents

#### Bibliotecas

In [2]:
import pandas as pd

#### Tratamento de Base
Redução do tamanho do arquivo a ser trabalhado

In [ ]:
df = pd.read_csv('US_Accidents_March23.csv')

# Converter 'Start_Time' para datetime, caso ainda não esteja
df['Start_Time'] = pd.to_datetime(df['Start_Time'])

# Filtrar os dados para os anos entre 2021 e 2023
filtered_df = df[(df['Start_Time'].dt.year >= 2021) & (df['Start_Time'].dt.year <= 2023)]

# Salvar a tabela filtrada em um novo arquivo CSV (ou trabalhar diretamente com filtered_df)
filtered_df.to_csv('US_Accidents_2021_2023.csv', index=False)

print("Tabela menor criada com sucesso!")

#### Contulta de Endereço via API

Pegar informação de Rua, Bairro, Cidade, Estado e País

In [ ]:
# A fazer

#### Estados com Maiores Casos